In [2]:
import numpy as np
import pandas as pd
import pdb
import time
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
from torch.distributions.multivariate_normal import MultivariateNormal as multivariate_normal
sns.set()
import gym
import simpleenvs # a custom mujoco env 
from kwng_es import *
from es_utils import *

In [3]:
class random_wassdist(nn.Module):
    
    
    def __init__(self, params: dict) -> None:
        super(random_wassdist, self).__init__()
        self.sigma = 1.0 
        self.gamma = 1.0 
        self.T = 10
        self.n = 1 # num data points per batch to update kernel function
        self.xdim = params['dim']
        self.alpha = 5e-2
        self.D = params['rf_dim'] # random feature dimension
        standard_normal = torch.distributions.normal.Normal(0, 1)
        self.omega = standard_normal.sample((self.D, self.xdim)) * 1.0 / self.sigma
        self.bias = standard_normal.sample((self.D, 1)) * 2 * np.pi
        self.t = 0
        
    def get_random_feature(self, x: torch.Tensor) -> torch.Tensor:
        """
        args:
            x: tensor of shape [xdim, ]
        """
        if type(x) != torch.Tensor: x = torch.tensor(x, dtype=torch.float32); 
        return torch.cos(torch.matmul(self.omega, x) + self.bias) * np.sqrt(2. / self.D)
    
    def update(self, X: torch.Tensor, Y: torch.Tensor, params: dict) -> None:
        """
        sample and update behavioral test functions using single_update
        """
        for _ in range(params['w_iter']):
            x = X[int(np.random.uniform() * len(X))]
            y = Y[int(np.random.uniform() * len(X))]
            self.single_update(x, y)
        
    def single_update(self, x: torch.Tensor, y: torch.Tensor) -> None:
        """
        update behavioral test functions
        """
        x = x.unsqueeze(1)
        y = y.unsqueeze(1)
        if self.t == 0:
            # initialize the functions
            self.beta_1 = self.get_random_feature(x).view(-1)
            self.beta_2 = self.get_random_feature(y).view(-1)
        else:
            # map to random feature space
            zx = self.get_random_feature(x)
            zy = self.get_random_feature(y)
            
            C = torch.sum((x - y).pow(2), axis=0)
            coeff = torch.exp((torch.matmul(self.beta_1, zx) - torch.matmul(self.beta_2, zy) - C) / self.gamma)
            weight = torch.tensor([1 - coeff])
            weight = weight.unsqueeze(0)
            
            # update the functions
            self.beta_1 += self.alpha * torch.mean(weight * zx, axis=1)
            self.beta_2 += self.alpha * torch.mean(weight * zy, axis=1)
        self.t += 1
        
    def wd(self, x: torch.Tensor, y: torch.Tensor) -> float:
        """
        compute the Wasserstein distance
        """
        x = x.unsqueeze(1)
        y = y.unsqueeze(1)
        
        zx = self.get_random_feature(x)
        zy = self.get_random_feature(y)
        beta1_zx = torch.matmul(self.beta_1, zx)
        beta2_zy = torch.matmul(self.beta_2, zy)
        wd = beta1_zx - beta2_zy \
            + (1 / self.gamma) \
            * torch.exp((beta1_zx - beta2_zy - torch.sum((x - y).pow(2))) / self.gamma) 
        return wd


In [36]:
def toeplitz(c: torch.Tensor, r: torch.Tensor) -> torch.Tensor:
    """
    (slow) pytorch implementation of scipy's toeplitz function
    args:
        c: the first column of the toeplitz matrix
        r: the first row; note that differences in the top left entry 
            default to the first entry of the first column
    outputs:
        the result toeplitz matrix
    """
    c = c.view(-1); r = r.view(-1); 
    num_rows = len(c); num_cols = len(r); 
    m = torch.zeros([num_rows, num_cols], dtype=torch.float32)
    m[0, :] = r
    m[:, 0] = c
    
    # the final num_cols - 1 entries in row r are the first 
    # num_cols - 1 entries of the row above
    # this results in a matrix with constant diagonals
    for r in range(1, num_rows):
        m[r, 1:] = m[r-1, :-1]
    return m

class ToeplitzPolicy(nn.Module):
    
    def __init__(self, args: dict) -> None:
        super(ToeplitzPolicy, self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.init_seed = args['seed']
        self.ob_dim = args['ob_dim']
        self.h_dim = args['h_dim']
        self.ac_dim = args['ac_dim']
        
        self.w1 = nn.Parameter(
            self.weight_init(self.ob_dim + self.h_dim -1, args['zeros'])
        )
        
        self.b1 = nn.Parameter(
            self.weight_init(self.h_dim, args['zeros'])
        )

        self.w2 = nn.Parameter(
            self.weight_init(self.h_dim * 2 - 1, args['zeros'])
        )

        self.b2 = nn.Parameter(
            self.weight_init(self.h_dim, args['zeros'])
        )
        self.w3 = nn.Parameter(
            self.weight_init(self.ac_dim + self.h_dim - 1, args['zeros'])
        )
        
        self.W1 = self.build_layer(self.h_dim, self.ob_dim, self.w1)
        self.W2 = self.build_layer(self.h_dim, self.h_dim, self.w2)
        self.W3 = self.build_layer(self.ac_dim, self.h_dim, self.w3)

        self.params = torch.cat([self.w1, self.b1, self.w2, self.b2, self.w3])
        self.N = len(self.params)
        
    
    def weight_init(self, d: int, zeros: bool) -> torch.Tensor:
        if zeros: w = torch.zeros(d, device=self.device, requires_grad=True); 
        else:
            torch.manual_seed(self.init_seed)
            w = torch.rand(d, device=self.device, requires_grad=True) / np.sqrt(d)
        return (w)
    
    def build_layer(self, d1: int, d2: int, v: torch.Tensor) -> torch.Tensor:
        # len v = d1 + d2 - 1
        col = v[:d1]
        row = v[(d1-1):]
        
        W = toeplitz(col, row) # sketchy 
        return (W)
    
    
    def update(self, vec: torch.Tensor) -> None:
        """add vector of updates to parameters"""
        
        with torch.no_grad():
            self.params += vec

            self.w1 += vec[:len(self.w1)]
            vec = vec[len(self.w1):]
            self.b1 += vec[:len(self.b1)]
            vec = vec[len(self.b1):]
            self.w2 += vec[:len(self.w2)]
            vec = vec[len(self.w2):]
            self.b2 += vec[:len(self.b2)]
            vec = vec[len(self.b2):]
            self.w3 += vec

            self.W1 = self.build_layer(self.h_dim, self.ob_dim, self.w1)
            self.W2 = self.build_layer(self.h_dim, self.h_dim, self.w2)
            self.W3 = self.build_layer(self.ac_dim, self.h_dim, self.w3)

    
    def __call__(self, X: torch.Tensor) -> torch.Tensor:

        z1 = torch.tanh(torch.matmul(self.W1, X) + self.b1)
        z2 = torch.tanh(torch.matmul(self.W2, z1) + self.b2)
        return (torch.tanh(torch.matmul(self.W3, z2)))
            


In [37]:
class worker(object):
    
    def __init__(self, params: dict, master: ToeplitzPolicy, A: torch.Tensor, i: int, train: bool = True) -> None:
        self.env = gym.make(params['env_name'])
        self.v = A[i, :] # the perturbation we will use

        params['zeros'] = True # initialize policy with zeros so we can set it to the current policy
        self.policy = ToeplitzPolicy(params)
        self.policy.update(master.params)
        self.timesteps = 0
        
    def do_rollouts(self, seed: int = 0, train: bool = True) -> None:
        
        self.policy.update(self.v) # Add the perturbation, to calculate F(theta + sigma * epsilon)
        up, up_data = self.rollout(seed, train)
        
        self.policy.update(-2 * self.v) # Subtract the perturbation, to calculate F(theta - sigma * epsilon)
        down, down_data = self.rollout(seed, train)
        
        self.rewards = torch.tensor([up, down]).view(2)
        self.up_data = up_data
        self.down_data = down_data
    
    def rollout(self, seed: int = 0, train: bool = True) -> tuple:
        self.env.seed(0)
        state = self.env.reset()
        self.env._max_episode_stesp = 400 
        total_reward = 0
        done = False
        data = []
        while not done:
            action = self.policy(torch.tensor(state, dtype=torch.float32))
            action_dist = multivariate_normal(action, 0.01 * torch.eye(action.numel()))
            action = action_dist.sample((1,))
            state, reward, done, _ = self.env.step(action.numpy())
            total_reward += reward
            data.append([state, reward])
            self.timesteps += 1
        return (total_reward, data)

In [38]:
def embed(data: list) -> torch.Tensor:
    # reward to go - used for ant-wall env
    rewards = [x[1] for x in data]
    d = pd.DataFrame({'Traj': rewards[::-1]})
    embedding = torch.tensor(d.cumsum().Traj.values, dtype=torch.float32)
    return(embedding)

def calcdists(embeddings: list, dists: torch.Tensor, i: int, master: ToeplitzPolicy,
              m_embedding: torch.Tensor, params: dict):
    
    n_master = min(params['n_iter'], params['n_prev'])

    if n_master== 1:
        dists[i, 0] = master.wass.wd(m_embedding[0], embeddings[i])
        dists[i, 1] = master.wass.wd(m_embedding[0], embeddings[i+params['num_sensings']])
    else:
        # if we are comparing vs multiple previous policies
        dists[i, 0] = torch.mean(torch.tensor([master.wass.wd(x, embeddings[i]) for x in m_embedding]))
        dists[i, 1] = torch.mean(torch.tensor([master.wass.wd(x, embeddings[i+params['num_sensings']]) for x in m_embedding]))
    return(dists)

In [39]:
def aggregate_rollouts(master: ToeplitzPolicy, A: torch.Tensor, params: dict) -> tuple:
       
    all_rollouts = torch.zeros([params['num_sensings'], 2])
    up, down = [],[]
    timesteps = 0
    
    for i in range(params['num_sensings']):
        w = worker(params, master, A, i)
        w.do_rollouts()
        all_rollouts[i] = w.rewards
        up.append(embed(w.up_data))
        down.append(embed(w.down_data))
        timesteps += w.timesteps

    embeddings = up + down
    
    if params['optimizer'] == 'ES':
        dists = torch.zeros(params['num_sensings'])
    else:
        # Update behavioral test funcs and use them to calculate WDs for each perturbed policy
        if params['n_iter'] == 1:
            dists = torch.zeros(params['num_sensings'])
        else:
            dists = torch.zeros([params['num_sensings'], 2])
            master.wass.update(master.buffer, embeddings, params)
            for i in range(params['num_sensings']):
                dists = calcdists(embeddings, dists, i, master, master.embedding, params)

            # normalize dists
            dists = (dists - torch.mean(dists)) / (torch.std(dists)  + 1e-8)     
            dists = dists[:, 0] - dists[:, 1]
        master.buffer = embeddings
    
    # normalize rewards    
    all_rollouts = (all_rollouts - torch.mean(all_rollouts)) / (torch.std(all_rollouts)  + 1e-8)  
    # compute R_k - R_t
    m = all_rollouts[:, 0] - all_rollouts[:, 1]
    
    return(m, dists, torch.stack(embeddings), timesteps)


class ES_criterion(nn.Module):

    def __init__(self, beta=0.):
        """
        the ES loss function
        """
        super(ES_criterion, self).__init__()
        self.beta = beta
        
    def forward(self, perturbed_rewards, wdists, sample_params, param_dist):
        F = (1 - self.beta) * perturbed_rewards + self.beta * wdists
        return F.mean()

def ES(params: dict, master: ToeplitzPolicy) -> tuple:
    
    A_dist = multivariate_normal(torch.zeros(master.N), torch.eye(master.N))
    A = A_dist.sample((params['num_sensings'],))
    A *= params['sigma']
    A /= torch.norm(A, dim=-1).view(-1, 1)
        
    noisy_rewards, wdists, outputs, timesteps = aggregate_rollouts(master, A, params)
    
    g = torch.zeros(master.N)
    for i in range(params['num_sensings']):
        eps = A[i, :]
        # combine reward and WD
        g += eps * ((1 - params['beta']) * noisy_rewards[i] + params['beta'] * wdists[i]) # *
    
    g /= (2 * params['sigma'])
    
    return(g, noisy_rewards, wdists, outputs, A, A_dist, timesteps)

In [43]:
def Adam(dx, m, v, learning_rate, t, eps = 1e-8, beta1 = 0.9, beta2 = 0.999):
    m = beta1 * m + (1 - beta1) * dx
    mt = m / (1 - beta1 ** t)
    v = beta2 * v + (1-beta2) * (dx.pow(2))
    vt = v / (1 - beta2 **t)
    update = learning_rate * mt / (torch.sqrt(vt) + eps)
    return(update, m, v)

def SGD(dx, m, v, learning_rate, t):
    return learning_rate * dx

def train(params):
    torch.manual_seed(params['seed'])
    np.random.seed(params['seed'])
    
    if params['optimizer'] == 'ES':
        params['beta'] = 0 # ie no WD term in the blackbox function F
    else:
        params['beta'] = 0.5 # we use equal weight for the reward and WD. Other schemes could be tested...
    
    env = gym.make(params['env_name'])
    params['ob_dim'] = env.observation_space.shape[0]
    params['ac_dim'] = env.action_space.shape[0]

    m = 0
    v = 0
        
    params['zeros'] = False
    master = ToeplitzPolicy(params)
    print (master.N // params['sensings'])
    
    master.wass = random_wassdist(params) # initialize the behavioral test functions
    master.buffer = []
    master.embedding = []
    
    # KWNG
    criterion = ES_criterion(beta=params['beta'])

    device = "cuda" if tr.cuda.is_available() else "cpu"
    wrapped_optimizer = get_wrapped_optimizer(params,
                                      criterion,
                                      master,
                                      device=device)
        
    n_iter = 1
    ts_cumulative = 0
    ts = []
    rewards = []
    all_weights = pd.DataFrame()
    start_time = time.time()
    clock_ts = [] 
    
    while n_iter < params['max_iter'] + 1:
            
        params['n_iter'] = n_iter
        
        # main calculations
        gradient, reward, wdists, outputs, noise, noise_dist, timesteps = ES(params, master) 
        
        ts_cumulative += timesteps
        ts.append(ts_cumulative)
        
        # modify the gradients, if desired, via the wrapper
        noisy_params = master.params.view(1, -1) + noise
        gradient = wrapped_optimizer.step(
            gradient, reward, wdists, outputs, noisy_params, noise_dist, params['sigma'], params['learning_rate']
        )
        
        # update policy
        gradient /= (torch.norm(gradient) / master.N + 1e-8)
        update, m, v = Adam(gradient, m, v, params['learning_rate'], n_iter)
        master.update(update)
        
        # evaluate new policy, keep the PPE to repel the next iteration
        test_policy = worker(params, master, np.zeros([1, master.N]), 0)
        reward, master_traj = test_policy.rollout()
        master.embedding.append(embed(master_traj)) #params,
        master.embedding = master.embedding[-params['n_prev']:]
        rewards.append(reward)
        clock_ts.append(time.time() - start_time)
        
        if n_iter % 1 == 0:
            print('Iteration: %s/%s, Reward: %s, tot. time: %s' % \
                  (n_iter, params['max_iter'], reward, clock_ts[-1]/60.))
        
        if n_iter % 5 == 0 and params['save']:
            ns = params['num_sensings'] 
            path = f"./checkpts/{params['optimizer']}_{params['estimator']}_{ns}ns_{params['env_name']}_{params['seed']}_"
            np.savetxt(path + "params.csv", master.params.detach().numpy(), delimiter=',')
            np.savetxt(path + "adam_m.csv", m.detach().numpy(), delimiter=',')
            np.savetxt(path + "adam_v.csv", v.detach().numpy(), delimiter=',')
            np.savetxt(path + "embeddings.csv", tr.stack(master.embedding).detach().numpy(), delimiter=',')
            df = pd.DataFrame({
                'Optimizer': [params['optimizer'] for _ in range(len(rewards))],
                'Reward': rewards,
                'Timesteps': ts,
                'Clock_time': clock_ts
            })
            clip = "Clip" if params['clip_grad'] else "" #_{ns}ns
            f = f"./saved/{params['optimizer']}_{params['estimator']}{clip}_{ns}ns_antwall-v0_{params['seed']}.pkl"
            df.to_pickle(f)
          
        n_iter += 1
        
    df = pd.DataFrame({
        'Optimizer': [params['optimizer'] for _ in range(len(rewards))],
        'Reward': rewards,
        'Timesteps': ts,
        'Clock_time': clock_ts
    })
    return(df, rewards, ts)


In [44]:
params = {}
params['env_name'] = 'antwall-v0'
params['max_iter'] = 200 # number of iterations, >100 is usually enough
params['sensings'] = 2 # population size = num params / sensings
params['sigma'] = 0.12 # scale for the samples
params['h_dim'] = 64 # size of neural network hidden layers
params['rf_dim'] = 400 # dimensionality of random features, higher = more accurate but slower
params['dim'] = params['rf_dim']
params['w_iter'] = 100 # number of samples to update behavioral test funcs
params['n_prev'] = 2 # number of previous policies to compute WD vs current. We have an ablation of this in Fig
################# WNG stuff #####################
params['epsilon'] = 1e-5
params['num_basis'] = 5 # cannot be more than 2x population size 
params['with_diag_mat'] = 1
params['dumping_freq'] = 5
params['reduction_coeff'] = 0.85
params['min_red'] = 0.25
params['max_red'] = 0.75
params['base_optimizer'] = 'sgd'
params['kernel'] = 'gaussian'
params['log_bandwidth'] = 0.
params['momentum'] = 0.
params['weight_decay'] = 0.
params['basis_schedule'] = {} #{30: 1, 60: 5}


In [45]:
params['optimizer'] = 'ES' # BGES or ES
params['estimator'] = 'KWNG' # KWNG or EuclideanGradient
params['save'] = False
params['kwng_beta'] = -1.3 
params['clip_grad'] = False
params['num_sensings'] = 251
params['seed'] = 1 
params['learning_rate'] = 0.02
df, rewards, timesteps = train(params)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


251
KWNG beta = -1.3
sigma:   28638.94140625
Iteration: 1/200, Reward: -6102.391999192827, tot. time: 2.759707430998484
sigma:   54340.3203125
Iteration: 2/200, Reward: -6117.0509994583135, tot. time: 5.647935716311137
sigma:   238140.0625
Iteration: 3/200, Reward: -6075.880726920412, tot. time: 8.430505279699961


KeyboardInterrupt: 

In [ ]:
params['optimizer'] = 'ES'
params['estimator'] = 'EuclideanGradient'
params['save'] = False
params['kwng_beta'] = -1.3 
params['clip_grad'] = False
params['num_sensings'] = 251
params['seed'] = 1 # 2 is slow for KWNG w/ 1.3
params['learning_rate'] = 0.02

all_rewards = np.zeros([5, 200])
for seed in [0, 1, 2, 3, 4]:
    print ("==========================================")
    print (f"training seed {seed}")
    params['seed'] = seed
    df, rewards, timesteps = train(params)
    all_rewards[seed, :] = np.array(rewards)

In [ ]:
def errorfill(x, y, yerr, color='C0', alpha_fill=0.3, alpha_line=1.0, ax=None, label=None, lw=1):
    ax = ax if ax is not None else plt.gca()
    if np.isscalar(yerr) or len(yerr) == len(y):
        ymin = y - yerr
        ymax = y + yerr
    elif len(yerr) == 2:
        ymin, ymax = yerr
    ax.plot(x, y, color=color, label=label, lw=lw, alpha=alpha_line)
    #ax.set_ylabel(r'fitness')
    #ax.set_xlabel(r'evaluations')
    ax.tick_params(axis='both', labelsize=20)
    ax.grid(alpha=0.7)
    ax.legend(fontsize=22)
    ax.fill_between(x, ymax, ymin, color=color, alpha=alpha_fill)

In [ ]:
plt.plot(df.Timesteps, df.Reward, label='ES')
plt.xlabel("Timesteps")
plt.ylabel("Reward")
plt.legend(loc="lower right")
plt.show()